In [1]:
!pip install lxml requests
from google.colab import drive
drive.mount('/content/drive')

!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive'
!pip install chromedriver-autoinstaller
!pip install selenium
!pip install langchain

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 44 not upgraded.


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
from langchain.schema import Document
import uuid

# Document 클래스 정의
class Document:
    def __init__(self, page_content, metadata):
        self.id = str(uuid.uuid4())
        self.page_content = page_content
        self.metadata = metadata

# Chrome 옵션 설정 (헤드리스 모드로 실행)
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

documents = []

# 웹 드라이버 초기화
driver = webdriver.Chrome(options=chrome_options)

urls = [
    'https://product.kyobobook.co.kr/detail/S000213800371',      # 당신이 누군가를 죽였다.
    'https://product.kyobobook.co.kr/detail/S000213567877',      # 동경
    'https://product.kyobobook.co.kr/detail/S000213682152',      # 온전히 평등하고 지극히 차별적인
    'https://product.kyobobook.co.kr/detail/S000213900198',      # 쌤쌤 쿡북
    'https://product.kyobobook.co.kr/detail/S000213848131',      # 저속노화 식사법
    'https://product.kyobobook.co.kr/detail/S000213133026',      # 하나님, 그래서 그러셨군요!
    'https://product.kyobobook.co.kr/detail/S000213740560',      # 전지적 루이 & 후이 시점
    'https://product.kyobobook.co.kr/detail/S000213368065',      # THE MONEY BOOK(더 머니북)
    'https://product.kyobobook.co.kr/detail/S000213495686',      # 나만의 GPTs 앱으로 생산성 10배 늘리기
    'https://product.kyobobook.co.kr/detail/S000212058462',      # 디스 이즈 도쿄(2024~2025)
    'https://product.kyobobook.co.kr/detail/S000213513428',      # 부모의 어휘력
    'https://product.kyobobook.co.kr/detail/S000213641159',      # 마녀와의 7일
    'https://product.kyobobook.co.kr/detail/S000213641247',      # 탕비실
    'https://product.kyobobook.co.kr/detail/S000213835380',      # 역사의 쓸모
    'https://product.kyobobook.co.kr/detail/S000213690458',      # 무서운 그림들
    'https://product.kyobobook.co.kr/detail/S000213674078',      # 디자이너의 유학
    'https://product.kyobobook.co.kr/detail/S000213719520',      # 출근길 지하철
]

# 페이지 로드
wait = WebDriverWait(driver, 10)

try:
    for url in urls:
        driver.get(url)
        print(f"Successfully retrieved the web page: {url}")

        # 제목 추출
        try:
            title_xpath = '//*[@id="contents"]/div[1]/div/div[1]/div/div[1]/div[1]/div/h1/span'
            title_element = wait.until(EC.presence_of_element_located((By.XPATH, title_xpath)))
            title = title_element.text
        except Exception as e:
            title = "N/A"
            print(f"Title not found for {url}: {e}")

        # 작가 이름 추출
        try:
            author_xpath = '//*[@id="contents"]/div[1]/div/div[2]/div/div[1]/div[1]/div[1]/div[1]/div/div'
            author_element = wait.until(EC.presence_of_element_located((By.XPATH, author_xpath)))
            author = author_element.text.strip()
        except Exception as e:
            author = "N/A"
            print(f"Author not found for {url}: {e}")

        # 목차 추출
        try:
            toc_xpath = '//*[@id="scrollSpyProdInfo"]/div[10]/div[2]/div[1]/div/ul/li'
            toc_elements = wait.until(EC.presence_of_all_elements_located((By.XPATH, toc_xpath)))
            toc_content = [item.text.strip() for item in toc_elements]
            toc = "\n".join(toc_content)
        except Exception as e:
            toc = "N/A"
            print(f"Table of contents not found for {url}: {e}")

        # 책 속으로 추출
        try:
            book_intro_xpath = '//*[@id="scrollSpyProdInfo"]/div[11]/div[2]/div[1]/div/p'
            book_intro_elements = wait.until(EC.presence_of_all_elements_located((By.XPATH, book_intro_xpath)))
            book_intro_content = [item.text.strip() for item in book_intro_elements]
            book_intro = "\n".join(book_intro_content)
        except Exception as e:
            book_intro = "N/A"
            print(f"Book intro not found for {url}: {e}")

        # 출판사 서평 추출
        try:
            publisher_review_xpath = '//*[@id="scrollSpyProdInfo"]/div[12]/div[2]/div[1]/div/p'
            publisher_review_elements = wait.until(EC.presence_of_all_elements_located((By.XPATH, publisher_review_xpath)))
            publisher_review_content = [item.text.strip() for item in publisher_review_elements]
            publisher_review = "\n".join(publisher_review_content)
        except Exception as e:
            publisher_review = "N/A"
            print(f"Publisher review not found for {url}: {e}")

        # 리뷰 추출
        try:
            review_selector = (
                'div.comment_list > div.comment_item > div.comment_contents '
                '> div.comment_contents_inner > div.comment_view_wrap '
                '> div.comment_text_box > div.comment_text'
            )
            review_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, review_selector)))
            review_content = [item.text.strip() for item in review_elements]
            review_str = "\n".join(review_content)
        except Exception as e:
            review_str = "N/A"
            print(f"Reviews not found for {url}: {e}")

        page_content = f"제목:{title}\n\n작가 이름: {author}\n\n목차: {toc}\n\n책 속으로: {book_intro}\n\n출판사 서평: {publisher_review}\n\n리뷰: {review_str}"
        # Document 객체 생성 및 추가
        doc = Document(
            page_content=page_content,
            metadata={
                'title': title,
                'author': author,
                'table_of_contents': toc,
                'book_intro': book_intro,
                'publisher_review': publisher_review,
                'review': review_str,
                'url': url
            }
        )
        documents.append(doc)

finally:
    # 드라이버 종료
    driver.quit()

# 생성된 Document 객체 출력 (검토용)
for doc in documents:
    print("ID:", doc.id)
    print("Title:", doc.metadata['title'])
    print("Author:", doc.metadata['author'])
    print("Table of Contents:", doc.metadata['table_of_contents'])
    print("Book Intro:", doc.metadata['book_intro'])
    print("Publisher Review:", doc.metadata['publisher_review'])
    print("Review:", doc.metadata['review'])
    print("URL:", doc.metadata['url'])
    print("\n" + "="*50 + "\n")

Successfully retrieved the web page: https://product.kyobobook.co.kr/detail/S000213800371
Book intro not found for https://product.kyobobook.co.kr/detail/S000213800371: Message: 

Successfully retrieved the web page: https://product.kyobobook.co.kr/detail/S000213567877
Successfully retrieved the web page: https://product.kyobobook.co.kr/detail/S000213682152
Book intro not found for https://product.kyobobook.co.kr/detail/S000213682152: Message: 

Successfully retrieved the web page: https://product.kyobobook.co.kr/detail/S000213900198
Table of contents not found for https://product.kyobobook.co.kr/detail/S000213900198: Message: 

Book intro not found for https://product.kyobobook.co.kr/detail/S000213900198: Message: 

Publisher review not found for https://product.kyobobook.co.kr/detail/S000213900198: Message: 

Successfully retrieved the web page: https://product.kyobobook.co.kr/detail/S000213848131
Publisher review not found for https://product.kyobobook.co.kr/detail/S000213848131: Me

In [3]:
import pandas as pd

data = []
for doc in documents:
    data.append({
        'id': doc.id,
        'title': doc.metadata['title'],
        'author': doc.metadata['author'],
        'table_of_contents': doc.metadata['table_of_contents'],
        'book_intro': doc.metadata['book_intro'],
        'publisher_review': doc.metadata['publisher_review'],
        'review': doc.metadata['review'],
        'url': doc.metadata['url']
    })

df = pd.DataFrame(data)
df.to_csv('books_data.csv', index=False)

In [4]:
! pip install datasets

In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [7]:
from datasets import Dataset
import pandas as pd

df = pd.read_csv('books_data.csv')
dataset = Dataset.from_pandas(df)
dataset.push_to_hub("HyeoniLEE/books_dataset")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/533 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/HyeoniLEE/books_dataset/commit/94ea8555d652257ca042aaa9a85e4a0e49db9268', commit_message='Upload dataset', commit_description='', oid='94ea8555d652257ca042aaa9a85e4a0e49db9268', pr_url=None, pr_revision=None, pr_num=None)

In [8]:
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForMaskedLM
from datasets import load_dataset

# 데이터셋 로드
dataset = load_dataset("HyeoniLEE/books_dataset")

# 데이터셋을 훈련 세트와 검증 세트로 나누기
dataset = dataset["train"].train_test_split(test_size=0.1)  # 10%를 검증 세트로 사용

# 토크나이저 및 모델 로드
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")

# 전처리 함수 수정
def preprocess_function(examples):
    texts = []
    for i in range(len(examples["title"])):
        text = (
            (examples["title"][i] if examples["title"][i] is not None else "") + " " +
            (examples["author"][i] if examples["author"][i] is not None else "") + " " +
            (examples["table_of_contents"][i] if examples["table_of_contents"][i] is not None else "") + " " +
            (examples["book_intro"][i] if examples["book_intro"][i] is not None else "") + " " +
            (examples["publisher_review"][i] if examples["publisher_review"][i] is not None else "") + " " +
            (examples["review"][i] if examples["review"][i] is not None else "")
        )
        texts.append(text)
    tokenized_inputs = tokenizer(texts, truncation=True, padding="max_length", max_length=512)
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()  # labels 추가
    return tokenized_inputs

# 데이터셋 전처리
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# 학습 인자 설정
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

# 트레이너 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)

# 모델 학습
trainer.train()


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Generating train split:   0%|          | 0/17 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.140675
2,No log,0.107896
3,No log,0.099945


TrainOutput(global_step=12, training_loss=0.12621853748957315, metrics={'train_runtime': 447.5757, 'train_samples_per_second': 0.101, 'train_steps_per_second': 0.027, 'total_flos': 11844216576000.0, 'train_loss': 0.12621853748957315, 'epoch': 3.0})

In [9]:
!pip install huggingface_hub
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write)

In [11]:
new_model_path = "./book_finetuned_model"

# 모델 저장
trainer.save_model(new_model_path)
tokenizer.save_pretrained(new_model_path)

('./book_finetuned_model/tokenizer_config.json',
 './book_finetuned_model/special_tokens_map.json',
 './book_finetuned_model/vocab.txt',
 './book_finetuned_model/added_tokens.json',
 './book_finetuned_model/tokenizer.json')

In [21]:
import os
from pathlib import Path

# 모델 저장 경로
new_model_path = "./book_finetuned_model"

readme_path = Path(new_model_path) / "README.md"
readme_content = '이 모델은 QA task를 위해 bert를 파인튜닝한 모델입니다.'

## 사용 방법
'''
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForMaskedLM.from_pretrained("your-username/your-model-name")
tokenizer = AutoTokenizer.from_pretrained("your-username/your-model-name")'''

with open(readme_path, "w") as f:
    f.write(readme_content)

model = AutoModelForMaskedLM.from_pretrained(new_model_path)
tokenizer = AutoTokenizer.from_pretrained(new_model_path)

model.push_to_hub("HyeoniLEE/book_finetuned_model")
tokenizer.push_to_hub("HyeoniLEE/book_finetuned_model")

CommitInfo(commit_url='https://huggingface.co/HyeoniLEE/book_finetuned_model/commit/791e4f656dbae1a327b483b56d3e050766a4634c', commit_message='Upload tokenizer', commit_description='', oid='791e4f656dbae1a327b483b56d3e050766a4634c', pr_url=None, pr_revision=None, pr_num=None)

In [23]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
import torch

# 모델과 토크나이저 불러오기
model_name = "HyeoniLEE/book_finetuned_model"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 대화 함수
def get_chat_response(question):
    inputs = tokenizer.encode_plus(question, return_tensors='pt')
    input_ids = inputs['input_ids'].tolist()[0]

    with torch.no_grad():
        outputs = model(**inputs)
        answer_start_scores = outputs.start_logits
        answer_end_scores = outputs.end_logits

    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    return answer

if __name__ == "__main__":
    while True:
        user_question = input("당신이 원하는 책은?: ")
        if user_question.lower() in ['exit', 'quit', 'q']:
            break
        result = get_chat_response(user_question)
        print(f"Book_bot: {result}")

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at HyeoniLEE/book_finetuned_model and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


당신이 원하는 책은?: 최태성 작가의 역사에 관한 책을 알려줘
Book_bot: 작가의
당신이 원하는 책은?: 역사 책 중 최태성 작가의 책은?
Book_bot: 중 [UNK] 작가의 책은?


KeyboardInterrupt: Interrupted by user

In [24]:
from transformers import pipeline

# Hugging Face Hub에 업로드한 모델 이름
model_name = "HyeoniLEE/book_finetuned_model"

# QA 파이프라인 생성
qa_pipeline = pipeline("question-answering", model=model_name, tokenizer=model_name)

# 대화 함수
def get_chat_response(question, context):
    result = qa_pipeline(question=question, context=context)
    return result['answer']

if __name__ == "__main__":
    # 사전 정의된 컨텍스트
    some_context = """
    최태성 작가는 한국사에 대한 여러 책을 썼습니다. 그 중에서 '역사 속 숨은 이야기', '한국사 그날', '한눈에 읽는 한국사' 등이 있습니다.
    이 책들은 한국사의 중요한 사건과 인물을 다루고 있으며, 일반 독자들이 쉽게 이해할 수 있도록 쓰여졌습니다.
    """

    while True:
        user_question = input("당신이 원하는 책은?: ")
        if user_question.lower() in ['exit', 'quit', 'q']:
            break
        result = get_chat_response(user_question, some_context)
        print(f"Book_bot: {result}")

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at HyeoniLEE/book_finetuned_model and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


당신이 원하는 책은?: 최태성 작가의 책을 알려줘
Book_bot: 숨은 이야기'
당신이 원하는 책은?: 최태성 작가의 한국사 책은?
Book_bot: 중에서


KeyboardInterrupt: Interrupted by user